In [1]:
max1demand=10    #use 1 more than actual demand
max2demand=10    #use 1 more than actual demand
sparti=[]  #start with null #ZoneActiveStateDefinition 1-Not active in 1; 2-Not active in 2; 3- Active in 1; 4- Active in 2;
spartj=[]  #start with null #Zone1DemandStateDef
spartk=[]  #start with null #Zone2DemandStateDef
spartl=[]  #start with null #PreviousZoneStateDef
sidx=[]    #start with null
s=1        #start number for loop
for i in 1:4           #Create an index to call   
    for j in 1:max1demand
        for k in 1:max2demand
            for l in 1:2 #2 regions
                sparti= push!(sparti,i)  #for i in ijk add to sparti definition
                spartj= push!(spartj,j)  #for j in ijk add to spartj definition
                spartk= push!(spartk,k)  #for k in ijk add to spartk definition
                spartl= push!(spartl,l)  #for k in ijk add to spartk definition
                sidx= push!(sidx,s)      #create index in sidx
                s=s+1
            end
        end
    end
end

#sparti
#spartj
#spartk
#[sparti,spartj,spartk]
#sidx

#idxactive=  (sparti.>=3) .& (spartj+spartk.==2) ##(Wrapped into regional indices)
idxidle= (sparti.<3) .& (spartj+spartk .>2)   #index all ijk where idling cannot occur
idxZoneFrom= (sparti.==1) .& (spartl.==2) .| (sparti.==2) .& (spartl.==1) #idx where zone from cannot occur
idxregion1= (sparti.==3) .& (spartj.==1)      #index all ijk where region 1 is active but there are no calls in region 1 to service
idxregion2= (sparti.==4) .& (spartk.==1)      #index all ijk where region 2 is active but there are no calls in region 2 to service


idxDEL=idxidle .| idxregion1 .| idxregion2 .| idxZoneFrom   #index of the previous 3 indices

deleteat!(sparti, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartj, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartk, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartl, BitArray(idxDEL)) #delete idxDEL
sidx= [1:length(sparti);];          #number of states
sidx

362-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
 351
 352
 353
 354
 355
 356
 357
 358
 359
 360
 361
 362

In [2]:
λ=1.75
μ1=5
μ2=4
μMatr=[μ1 μ2; 
       μ2 μ1]

2×2 Matrix{Int64}:
 5  4
 4  5

In [3]:
N=["λ" "μ1" "μ2" "Throughput" "Deliveries/Hour" "Avg. # in Sys" "Avg. Time in Sys" "% of Time 0 calls" "% of Time 1 call" ]
λ=0
μ1=.75
μ2=.75

for i in 1:4
    λ=λ+.125
    μ1=.75
    for j in 1:17
      μ1=μ1+.25
      μ2=.75
        for k in 1:17
          μ2=μ2+.25
            if  μ2>μ1
                break
            end
μMatr=[μ1 μ2; 
       μ2 μ1]
matrix= zeros(length(sidx),length(sidx))
μMat= zeros(length(sidx),1)
λidx=zeros(length(sidx)) 
μidx=[]
λ1=λ  #Define Arrival Rate for Zone1
λ2=λ1  #Define Arrival Rate for Zone2                

for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    RegionFrom= spartl[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nexts=0
    RegionService= 0    
    nextRegionFrom=0

#  Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2
        if Zoneflag <=2
            nextZoneflag = 3 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag = Zoneflag  #don't use double equal
            nextRegionFrom=RegionFrom
        end

        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].=matrix[nexts,s] .+ λ
        matrix[s,s]= matrix[s,s] - λ
         λidx[s]=1
    end
    #Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1       
      if Zoneflag <=2
            nextZoneflag = 4 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag = Zoneflag  #don't use double equal
            nextRegionFrom=RegionFrom
        end
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].=matrix[nexts,s] .+ λ2
        matrix[s,s]= matrix[s,s] - λ2
        λidx[s]=1
    end
    
         #Service from Region to Region
    if Zoneflag == 3
        if Call1-1 >=max(Call2)/2 && Call1-1 >1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=3
            nextCall1=Call1-1
            nextCall2=Call2
            nextRegionFrom=1
            μidx=push!(μidx,1)        
        elseif Call1-1 >max(Call2)/2 && Call1-1 ==1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1-1
            nextCall2=Call2
            nextRegionFrom=1
            μidx=push!(μidx,1)            
        else
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag= 4
            nextCall1= Call1-1
            nextCall2= Call2
            nextRegionFrom=1
            μidx=push!(μidx,2)
        end    
    elseif Zoneflag == 4
        if Call2-1 >=max(Call1)/2 && Call2-1 >1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=4
            nextCall1=Call1
            nextCall2=Call2-1
            nextRegionFrom=2
            μidx=push!(μidx,1)           
        elseif Call2-1 >max(Call1)/2 && Call2-1 ==1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=2
            nextCall1=Call1
            nextCall2=Call2-1
            nextRegionFrom=2
            μidx=push!(μidx,1)
        else
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag= 3
            nextCall1= Call1
            nextCall2= Call2-1  
            nextRegionFrom=2
            μidx=push!(μidx,2)
        end
    else 
        RegionService= 0
        nextZoneflag=Zoneflag
        nextCall1=Call1
        nextCall2=Call2
        nextRegionFrom=RegionFrom
        μidx=push!(μidx,0)
    end    
  idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .&(spartl.==nextRegionFrom)
  nexts= sidx[idx] 
  matrix[nexts,s]=matrix[nexts,s].+RegionService
  matrix[s,s]= matrix[s,s].- RegionService   
    μMat[s,1]= μMat[s,1] .+ RegionService
 end
matrix[length(sidx),:].= 1 #prep Ahat for matrix inversion
inverse=inv(matrix) #invert Ahat matrix
X=inverse[:,length(sidx)] #extract π from matrix
X=round.(X,digits=10) #round to 10 digits
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
M=hcat(sidx,A,B,C,D,X)     #create readable matrix for state, sparti,spartj,spartk, and pi
Throughput=(λ)*sum(M[λidx.==1,6])*2
Gain=μMat'*X
L=0
for s in 1:length(sidx)
  L=L+(B[s]+C[s]-2)*X[s]
end
W=L/Throughput 
#Percentage of time with zero calls
Midx0= (spartj .+ spartk).==2
ZeroCall=sum(M[Midx0.==1,6])
#Percentage of time with one calls
Midx1= (spartj .+ spartk).==3
OneCall=sum(M[Midx1.==1,6])
N=[N; λ μ1 μ2 Throughput Gain L W ZeroCall OneCall]
        end
    end
end            

In [4]:
N

613×9 Matrix{Any}:
  "λ"    "μ1"   "μ2"   "Throughput"  …   "% of Time 1 call"
 0.125  1.0    1.0    0.25              0.1875
 0.125  1.25   1.0    0.25              0.174105
 0.125  1.25   1.25   0.25              0.16
 0.125  1.5    1.0    0.25              0.163962
 0.125  1.5    1.25   0.25           …  0.149573
 0.125  1.5    1.5    0.25              0.138889
 0.125  1.75   1.0    0.25              0.156104
 0.125  1.75   1.25   0.25              0.141525
 0.125  1.75   1.5    0.25              0.130724
 0.125  1.75   1.75   0.25           …  0.122449
 0.125  2.0    1.0    0.25              0.149866
 0.125  2.0    1.25   0.25              0.135151
 ⋮                                   ⋱  
 0.5    5.0    2.25   1.0               0.212762
 0.5    5.0    2.5    1.0               0.205938
 0.5    5.0    2.75   1.0               0.199531
 0.5    5.0    3.0    1.0               0.193583
 0.5    5.0    3.25   1.0            …  0.188094
 0.5    5.0    3.5    1.0               0.183039
 0.5

In [5]:
using DataFrames
df= DataFrame(N[2:Int(length(N)/9),:], :auto)

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0.125,1.0,1.0,0.25,0.25,0.333333,1.33333,0.75,0.1875
2,0.125,1.25,1.0,0.25,0.25,0.288858,1.15543,0.775689,0.174105
3,0.125,1.25,1.25,0.25,0.25,0.25,1.0,0.8,0.16
4,0.125,1.5,1.0,0.25,0.25,0.262218,1.04887,0.792648,0.163962
5,0.125,1.5,1.25,0.25,0.25,0.22393,0.895721,0.816968,0.149573
6,0.125,1.5,1.5,0.25,0.25,0.2,0.8,0.833333,0.138889
7,0.125,1.75,1.0,0.25,0.25,0.244474,0.977896,0.804704,0.156104
8,0.125,1.75,1.25,0.25,0.25,0.206564,0.826258,0.829023,0.141525
9,0.125,1.75,1.5,0.25,0.25,0.182842,0.731366,0.84539,0.130724


In [12]:
using CSV
CSV.write("C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\MidpointAmbStats8FEB_1.csv",df)

"C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\MidpointAmbStats8FEB_1.csv"